In [1]:
import gym
import numpy as np
import gym_map_bro
import toy_agents

env = gym.make('map-bro-v0')

In [2]:
env.__myinit__()

In [3]:
# New batch of 5 lines, here represented by 5 random values
np.random.seed(0)
new_batch = np.random.uniform([10.0, 10.0, 10.0, 10.0, 10.0])
print(new_batch)

[5.06067846 3.5632957  4.57512962 5.09605135 6.18710681]


In [4]:
# Run the Q agent
lr = .9
y = .95
num_episodes = 100
Q, rList = toy_agents.delayed_reward_agent(env, new_batch, lr, y, num_episodes)
print(Q)

[[ 0.45180005 -4.85718514]
 [ 0.05124368 -1.64849026]
 [ 1.04489175 -2.77144844]
 [ 1.53060843 -3.92820543]
 [ 1.10612078 -6.18710681]
 [ 0.          0.        ]]


In [8]:
# Determine best actions for the batch
batch_actions = np.argmax(Q, axis=1)
print(batch_actions[0:5])

[0 0 0 0 0]


In [6]:
# Part I'm working on now: actually perform the actions